In [1]:
import pandas
import weave
weave.use_frontend_devmode()
from weave.ecosystem import openai
from weave.ecosystem import umap
from weave.ecosystem import hdbscan

In [2]:
raw_data = pandas.read_csv('/Users/shawn/datasets/wandb_export_2023-06-03T15_01_20.066-07_00.csv')

data = weave.save(weave.ops.dataframe_to_arrow(raw_data), 'data')
#data

In [3]:
from weave import ops_arrow
x = ops_arrow.to_arrow([1, 2, 3, 4, 5])
x[[0, 3]].to_pylist_raw()

[1, 4]

In [33]:
limited_data = data.limit(1005)
r = weave.ops.range(0, limited_data.count(), 1)
embeddings = openai.openai_embed(limited_data['question'], {"model": "text-embedding-ada-002"})
clusterable_projection = umap.umap_project(
            embeddings, {
                'n_neighbors': 30,
                'min_dist': 0,
                'n_components': 2,
                'random_state': 42
            }
        )
clusters = hdbscan.hdbscan_cluster(clusterable_projection, {
            'min_samples': 10,
            'min_cluster_size': 50
        })

mapped_res = r.map(lambda row, index: weave.ops.dict_(
    a=clusterable_projection[index],
    #b=clusterable_projection[index][1],
    #c=clusters[index]
))
res = weave.use(mapped_res)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [27]:
len(weave.use(clusterable_projection[5]))

2

In [23]:
res._arrow_data

-- is_valid: all not null
-- child 0 type: float
  [
    10.122554,
    10.8042755,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    ...
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null,
    null
  ]

In [4]:
weave.panels.Board(
    vars={
        'data': data,
        'limit': 1005,
        'limited_data': lambda data, limit: data.limit(limit),
        'embeddings': lambda limited_data: openai.openai_embed(limited_data['question'], {"model": "text-embedding-ada-002"}),
        'clusterable_projection': lambda embeddings: umap.umap_project(
            embeddings, {
                'n_neighbors': 30,
                'min_dist': 0,
                'n_components': 2,
                'random_state': 42
            }
        ),
        'clusters': lambda clusterable_projection: hdbscan.hdbscan_cluster(clusterable_projection, {
            'min_samples': 10,
            'min_cluster_size': 50
        }),
    },
    panels=[
        weave.panels.BoardPanel(
            lambda limited_data, clusterable_projection, clusters: weave.panels.Plot(
                weave.ops.range(0, limited_data.count(), 1),
                #limited_data,
                x=lambda row, index: clusterable_projection[index][0],
                y=lambda row, index: clusterable_projection[index][1],
                label=lambda row, index: clusters[index],
            )
        )
    ]
)
#pass